# Variational Autoencoders
### Faustine Li and Lou Wang 

--------------------

### Abstract

### 1. Introduction

### 2. Description of Algorithm

### 4. Optimization

### 5. Applications

### 6. Comparison

### 7. Conclusion

### References 